# UCS Manager Python SDK Examples

---

## Connect to UCS

In [ ]:
from ucsmsdk.ucshandle import UcsHandle

UCS_CONN = {
  'ip': '192.168.72.4',
	'username': 'admin',
	'password': 'admin',
  'secure': False
}

handle = UcsHandle(**UCS_CONN)
handle.login()

---

## Review handle object attributes

In [ ]:
vars(handle)

handle.ip
handle.ucs
handle.cookie

---

## Query and display all compute blade objects

In [ ]:
blades = handle.query_classid('computeBlade')

for blade in blades:
  print(blade)

---

## Display specific blade attributes

In [ ]:
for blade in blades:
  print(blade.dn, blade.num_of_cpus, blade.available_memory)

---

## SDK Query Methods

### Get objects from a single class

Returns a list of objects: `query_classid`

In [ ]:
handle.query_classid('computeBlade')

---

### Get objects from multiple classes

Return a dictionary where `classid` is the key and objects are values: `query_classids`

In [ ]:
handle.query_classids('computeBlade', 'computeRackUnit')

---

### Get a single object by DN

Return an object: `query_dn`

In [ ]:
handle.query_dn('sys/chassis-3/blade-1')

---

### Get multiple objects by DN

Return a dictionary where the DN is the key and the object is the value: `query_dns`

In [ ]:

handle.query_dn('sys/chassis-3/blade-1', 'sys/chassis-3/blade-3')

---

### Get child objects for a specific object

Return a list of child objects: `query_children`

In [ ]:
handle.query_children(handle.query_dn('sys/chassis-3/blade-1'))

---

## Get Server DN and LED State

In [ ]:
# Get all server objects, using classes
servers = handle.query_children('computeBlade', 'computeRackUnit')

# Loop over all servers
for server_class in servers.values():
    # Loop over each server class object
    for server in server_class:
      # Assign the equipmentLocatorLed class data to a variable
      led = handle.query_children(server, class_id='equipmentLocatorLed')
      print(server.dn, led[0].oper_state)

---

## Chanege the state of all LEDs

In [ ]:
# Get a list of all server objects, by class
# Each class becomes its own dictionary key
server_classes = handle.query_classids('computeBlade', 'computeRackUnit')

# Loop over each server class object list (computeBlade and computeRackUnit)
for server_list in server_classes.values():
      # Loop over each server object within the list
      for server in server_list:
        # Get server LED status
        led = handle.query_children(server, class_id='equipmentLocatorLed')
        # Set previous LED status
        prev_led_status = led[0].oper_state

        # Set the new LED status
        if led[0].oper_state == 'off':
            led[0].admin_state = 'on'
        else:
          led[0].admin_state = 'off'

        # Set the handle MO value and commit the change
        handle.set_mo(led[0])
        handle.commit()
        
        # Display output
        print(f'DN {server.dn}:\n'
              f'\tPrevious - {prev_led_status}\n'
              f'\tNew - {led[0].admin_state}')

---

## Query the locator LED status for all servers, reverse the setting, and display results

In [ ]:
# Get lists of all servers (returns a dict)
servers = handle.query_classids('computeBlade', 'computeRackUnit')

# Loop over each of the dict values (separate lists of server classes)
for server_classes in servers.values():
  # Loop over each individual server class

  for server in server_classes:
  	# Get the LED subclass status for each server
	  led = handle.query_children(server, class_id='EquipmentLocatorLed')

    # Get the current LED properties
    led_dn = led[0].dn
    led_oper_state = led[0].oper_state

    # Reverse the LED state
    if led_oper_state == 'off':
    	led[0].admin_state = 'on'
    else:
      led[0].admin_state = 'off'
     
		# Set the new LED state
    handle.set_mo(led[0])
    handle.commit()

    # Get the updated status
    led_new_props = handle.query_dn(led_dn)

    # Display the results
    print(f'Locator LED State for {server.dn}:\n'
          f'\tPrevious state - {led_oper_state}\n'
          f'\tNew state - {led_new_props.admin_state}')

---

## Create a new VLAN

In [ ]:
# Import the UcsHandle and FabricVlan modules
from ucsmsdk.ucshandle import UcsHandle
from ucsmsdk.mometa.fabric.FabricVlan import FabricVlan

# Connect to the UCSM
UCSM_CONN = {
  'ip': '192.168.72.4',
  'username': 'admin',
  'password': 'admin',
  'secure': False
}

handle = UcsHandle(**UCSM_CONN)
handle.login()

# Get the FabricLanCloud class
lan_cloud = handle.query_classid('FabricLanCloud')

# Display the lan_cloud DN
# lan_cloud[0].dn

# Create a MO for the VLAN
vlan_mo = FabricVlan(lan_cloud[0].dn, name='vlan100', id='100')

# Add the VLAN MO to the UCS
handle.add_mo(vlan_mo)
# handle.add_mo(mo=vlan_mo, modify_present=True)
handle.commit()

---

## Create multiple VLANs in a transaction

In [ ]:
# Import the UcsHandle and FabricVlan modules
from ucsmsdk.ucshandle import UcsHandle
from ucsmsdk.mometa.fabric.FabricVlan import FabricVlan

# Connect to the UCSM
UCSM_CONN = {
  'ip': '192.168.72.4',
  'username': 'admin',
  'password': 'admin',
  'secure': False
}

handle = UcsHandle(**UCSM_CONN)
handle.login()

# Create a FabricLanCloud MO
lan_cloud = handle.query_classid('FabricLanCloud')

# Loop over a range of VLAN numbers
for vlan in range (300, 303):
  # Create a MO for each VLAN
  vlan_mo = FabricVlan(
    parent_mo_or_dn=lan_cloud[0].dn,
    name=f'vlan{vlan}',
    id=str(vlan)
  )

  # Add each MO to the handle object
  handle.add_mo(vlan_mo)

# Commit the changes to UCSM
handle.commit()

---

## Delete multiple VLANs in a transaction

In [ ]:
# Import modules
from ucsmsdk.ucshandle import UcsHandle
from ucsmsdk.mometa.fabric.FabricVlan import FabricVlan

# Log in
CONN = {
  'ip': '192.168.72.4',
  'username': 'admin',
  'password': 'admin',
  'secure': False
}

handle = UcsHandle(**CONN)
handle.login()

# Create a MO for all VLANs
vlans_mo = handle.query_classid('FabricVlan')

# Loop over the list of VLANs (in the vlans_mo), and remove them, except for VLAN 1
for vlan_mo in vlans_mo:
  if vlan_mo.id != '1':
	  handle.add_mo(vlan_mo)

handle.commit()

---

## UCSM time zone and NTP settings

### Import UcsHandle

In [ ]:
from ucsmsdk.ucshandle import UcsHandle

# Login
UCSM = {
  'ip': '192.168.72.4',
  'username': 'admin',
  'password': 'admin',
  'secure': 'false'
}

handle = UcsHandle(**UCSM)
handle.login()

---

### Get time zones

In [ ]:
# Import the CommDateTime class
from ucsmsdk.mometa.comm.CommDateTime import CommDateTime

# Query the CommDateTime DN that is a reference to the 'Timezone-managed' object
timezone_mo = handle.query_dn('sys/svc-ext/datetime-svc')
# timezone_mo = handle.query_classid('CommDateTime') # Also works the same way

---

Get NTP

In [ ]:
# Import the CommNtpProvider class
from ucsmsdk.mometa.comm.CommNtpProvider import CommNtpProvider

# Create a MO for the NTP objects
ntp_mos = handle.query_classid('CommNtpProvider')

# Display NTP object details
for n in ntp_mos:
  print(n)

---

### Get DNS

In [ ]:
# Import classes and create MOs
from ucsmsdk.mometa.comm.CommDns import CommDns
dns_mo = handle.query_classid('CommDns')
print(dns_mo[0])

from ucsmsdk.mometa.comm.CommDnsProvider import CommDnsProvider
dns_mos = handle.query_classid('CommDnsProvider')
print(dns_mos[0])

---

### Set Time Zones

In [ ]:
# Create a time zone variable
tz = 'America/Chicago'

# Update the time zone MO with the new value
timezone_mo[0].timezone = tz

# Update the MO
handle.set_mo(timezone_mo[0])

### Add and remove NTP servers

In [ ]:
# Create NTP server veriable
ntp_server = '172.16.20.5'

# Create an NTP parent MO
datetime_mo = handle.query_dn('sys/svc-ext/datetime-svc')

# Create an NTP MO
ntp_mo = CommNtpProvider(
	datetime_mo,
  name='172.16.20.5'
)

# Add and commit the MO
handle.add_mo(
	mo=ntp_mo,
  modify_present=True
)
handle.commit()

# Get a new MO for the new NTP provider
ntp_class = handle.query_classid('CommNtpProvider')
ntp_mo = ntp_class[0]

# Remove the NTP provider
handle.remove_mo(ntp_mo)
handle.commit()

---

### Add and remove DNS servers

In [ ]:
# Get parent DNS service object
dns_svc_mo = handle.query_dn('sys/svc-ext/dns-svc')

# Create a DNS provider MO
dns_provider_mo = CommDnsProvider(dns_svc_mo, name='8.8.8.8')

# Add and commit the MO to UCSM
handle.add_mo(dns_provider_mo, True)
handle.commit()

# Remove the DNS Provider
dns_providers_mo = handle.query_classid('CommDnsProvider')

### Option #1, loop ###
# Loop over the DNS providers and remove the entry for 8.8.8.8
for dns_server in dns_providers_mo:
  if dns_server.name == '8.8.8.8':
    dns_server_mo = dns_server
    handle.remove_mo(dns_server_mo)
    handle.commit()
    
### Option #2, user a filter string in the query_classid method ###
dns_provider_mos = handle.query_classid(
  class_id='CommDnsProvider',
  filter_str='(dn, "sys/svc-ext/dns-svc/*", type="re")'
)

---

### Handle query filter

In [ ]:
handle.query_classid(
    class_id=None,
    filter_str=None,
    hierarchy=False,
    need_response=False,
    timeout=None,
)
""" Finds an object using it's class id.

    Args:
        class_id (str): class id of the object to be queried for.
        filter_str(str): query objects with specific property with specific value or pattern specifying value.

                (property_name, "property_value, type="filter_type")

                property_name: Name of the Property
                ## example, dn (without quotes) ##

                property_value: Value of the property (str or regular expression)
                ## example, "sys/svc-ext/dns-svc/*" (with quotes) ##

                filter_type: eq - equal to

                            ne - not equal to

                            ge - greater than or equal to

                            gt - greater than

                            le - less than or equal to

                            lt - less than

                            re - regular expression

                            ## example, "re" (with quotes) ##

                logical filter type: not, and, or
"""

---

### Backup UCS configuration

In [ ]:
# Import the backup_ucs class
from ucsmsdk.utils.ucsbackup import backup_ucs

# Perform the backup
backup_ucs(
  handle,
  'config-all',
  './',
  'py-sdk-config-all.xml'
)

In [ ]:
backup_ucs(
    handle,
    backup_type,
    file_dir,
    file_name,
    timeout_in_sec=600,
    preserve_pooled_values=False,
)
""" Docstring:
    backup_ucs helps create and download Ucs backups.

    Args:
        handle (UcsHandle): Ucs Connection handle
        backup_type (str): type of backup
                        i.e. fullstate/config-logical/config-system/config-all
        file_dir (str): directory to download ucs backup file to
        file_name (str): name for the backup file
                        (supported file extension are '.tar.gz' and '.xml')
        timeout_in_sec (number) : time in seconds for which method waits
                            for the backUp file to generate before it exits.
        preserve_pooled_values (boolean): True/False,
                                            False - by default

    Example:
        file_dir = "/home/user/backup"

        file_name = "config_backup.xml"

        backup_ucs(handle, backup_type="config-logical",
                    file_dir=file_dir, file_name=file_name)
"""

---

### Import UCS configuration

In [ ]:
# Import UCS configuration
from ucsmsdk.utils.ucsbackup import import_ucs_backup

# Import configuration
import_ucs_backup(
	handle,
  './',
  'py-sdk-config-all.xml',
  merge=True
)